HW1-PART1(MapReduce)-question 2

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


A) Using the MapReduce pattern, write a program that shows the five channel IDs with the most link exchange in the output.


In [ ]:
# Import necessary libraries
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder.appName("LinkExchange").getOrCreate()

# Read input data from file 
input_file = "data.txt"
channels = spark.sparkContext.textFile(input_file)

# Define map function
def map_channels(line):
    fields = line.split()
    channel = fields[0]
    links = fields[1:]
    return (channel, len(links))

# Define reduce function
def reduce_channels(links1, links2):
    return links1 + links2

# Apply MapReduce process
mapped_channels = channels.map(map_channels)
reduced_channels = mapped_channels.reduceByKey(reduce_channels)
sorted_channels = reduced_channels.sortBy(lambda x: x[1], ascending=False)

# Show top 5 channel IDs with most link exchange
top_channels = sorted_channels.take(5)
print("Top 5 Channels by Link Exchanges:")
for channel in top_channels:
    print("Channel ID:", channel[0], "Link Exchange:", channel[1])

# Stop SparkSession
spark.stop()


Top 5 Channels by Link Exchanges:
Channel ID: 859 Link Exchange: 1933
Channel ID: 5306 Link Exchange: 1741
Channel ID: 2664 Link Exchange: 1528
Channel ID: 5716 Link Exchange: 1426
Channel ID: 6306 Link Exchange: 1394


B) Find the number of channel ID link exchanges with numbers 1748, 5633 and 3469



In [ ]:
import os
# Set PYTHONHASHSEED environment variable
os.environ['PYTHONHASHSEED'] = '0'

# Create SparkSession
spark = SparkSession.builder.appName("LinkExchange").getOrCreate()

# Read input data from file 
input_file = "data.txt"
channels = spark.sparkContext.textFile(input_file)

# Create SparkSession
spark = SparkSession.builder.appName("LinkExchange").getOrCreate()

# Define map function
def map_channels(line):
    fields = line.split()
    channel = int(fields[0])
    links = [int(link) for link in fields[1:]]
    for link in links:
        yield (link, 1)
    yield (channel, 0)

# Define reduce function
def reduce_channels(links1, links2):
    return links1 + links2

# Apply MapReduce process
mapped_channels = channels.flatMap(map_channels)
reduced_channels = mapped_channels.reduceByKey(reduce_channels)

# Find number of link exchanges for specific channels
channel_ids = [1748, 5633, 3469]
for channel_id in channel_ids:
    num_links = reduced_channels.lookup(channel_id)
    if len(num_links) > 0:
        print("Channel ID:", channel_id, "Number of Link Exchanges:", num_links[0])
    else:
        print("Channel ID:", channel_id, "has no link exchanges.")

# Stop SparkSession
spark.stop()


Channel ID: 1748 Number of Link Exchanges: 130
Channel ID: 5633 Number of Link Exchanges: 30
Channel ID: 3469 Number of Link Exchanges: 119
